# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherPyCSV=pd.read_csv('../WeatherPy/Weather_Data.csv')
weatherPyCSV

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cape town,-33.93,18.42,62.01,82,75,3.36,ZA,1603592715
1,hermanus,-34.42,19.23,61.00,84,37,1.99,ZA,1603592890
2,geraldton,-28.77,114.60,93.20,14,0,17.22,AU,1603592890
3,mahebourg,-20.41,57.70,69.80,88,75,12.75,MU,1603592891
4,iqaluit,63.75,-68.51,24.80,79,90,3.36,CA,1603592891
...,...,...,...,...,...,...,...,...,...
548,witu,-2.39,40.44,78.80,83,1,8.55,KE,1603592996
549,macia,-25.03,33.10,64.38,97,1,1.16,MZ,1603592996
550,honiara,-9.43,159.95,87.80,74,20,8.05,SB,1603592915
551,manadhoo,5.77,73.38,83.10,73,100,10.38,MV,1603592997


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherPyCSV[["Lat", "Lng"]].astype(float)

humidity = weatherPyCSV["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weatherPyCSV["Humidity"].max().astype(float),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cleanedWeather=weatherPyCSV.dropna()
idealWeather=cleanedWeather.loc[(cleanedWeather["Max Temp"]>65) & (cleanedWeather["Max Temp"]<80)]
idealWeatherDF=idealWeather.loc[(idealWeather["Cloudiness"]<5) & (idealWeather["Wind Speed"]<5)]
idealWeatherDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,siyabuswa,-25.11,29.04,70.11,62,0,2.71,ZA,1603592895
71,mbuyapey,-26.20,-56.75,75.96,55,0,3.31,PY,1603592907
84,irbil,36.19,44.01,68.68,23,0,4.54,IQ,1603592909
138,lahij,13.17,44.58,73.81,52,0,3.09,YE,1603592920
158,abu dhabi,24.47,54.37,77.00,33,0,4.70,AE,1603592881
207,livramento,-3.03,-60.18,77.00,94,0,4.54,BR,1603592932
224,mapiripan,2.89,-72.13,76.30,86,0,3.67,CO,1603592935
253,jardim,-21.48,-56.14,74.53,85,1,1.97,BR,1603592941
341,san patricio,28.02,-97.52,70.00,63,1,3.65,US,1603592901
350,korhogo,9.45,-5.63,71.60,59,0,1.77,CI,1603592960


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=idealWeatherDF[["City","Lat","Lng","Country"]].reset_index()
hotel_df["Hotel Name"]=""
hotel_df

,index,City,Lat,Lng,Country,Hotel Name
0,25,siyabuswa,-25.11,29.04,ZA,
1,71,mbuyapey,-26.20,-56.75,PY,
2,84,irbil,36.19,44.01,IQ,
3,138,lahij,13.17,44.58,YE,
4,158,abu dhabi,24.47,54.37,AE,
5,207,livramento,-3.03,-60.18,BR,
6,224,mapiripan,2.89,-72.13,CO,
7,253,jardim,-21.48,-56.14,BR,
8,341,san patricio,28.02,-97.52,US,
9,350,korhogo,9.45,-5.63,CI,


In [24]:
params={"radius":5000,"types":"lodging","key":g_key}

for index, row in hotel_df.iterrows():
    params["location"]=f"{row['Lat']},{row['Lng']}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
        print(f"The hotel closest to  {hotel_df.loc[index, 'City'] } is {name_address['results'][0]['name']}.")
    except:
        hotel_df.loc[index, 'Hotel Name'] = "N/A"
        print(f"There is no hotel within 5000km of {hotel_df.loc[index, 'City']}.")

The hotel closest to  siyabuswa is Uluju properties.
The hotel closest to  mbuyapey is Cabaña AM.
The hotel closest to  irbil is Erbil International Hotel.
There is no hotel within 5000km of lahij.
The hotel closest to  abu dhabi is Beach Rotana.
The hotel closest to  livramento is ArteNamata.
The hotel closest to  mapiripan is Monserrate.
The hotel closest to  jardim is Hotel Estância.
The hotel closest to  san patricio is Motel 6 Sinton, TX.
The hotel closest to  korhogo is Hotel Le Recul.
The hotel closest to  bannu is Aamir Hotel Bannu.
The hotel closest to  ulundi is Ekhethelo Lodge.
The hotel closest to  tuxtepec is Hotel Tuxtepec.
The hotel closest to  harda is OYO 67581 Samaroh Parisar.
The hotel closest to  hit is Aboody Alhachem.
The hotel closest to  chokwe is Complexo Sonho Real.
The hotel closest to  portel is Pousada Avenida.
The hotel closest to  salalah is HAMDAN PLAZA HOTEL.
The hotel closest to  nampula is Tacab Residencial.
The hotel closest to  sao gabriel da cachoe

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))